## Часть 1. Пример проблемы использования метода Гаусса для решения СЛАУ

Перед вами простая реализация метода Гаусса для решения СЛАУ. Далее по коду представлены две СЛАУ $A_1x = b_1$ и $A_2x = b_2$, эквивалентные с точностью до перестановки строк. Эти СЛАУ решаются сначала пакетным методом, затем реализованным методом Гаусса. Для пакетного метода получается два одинаковых решения. Для метода Гаусса - два отличающихся решения. ЗАДАНИЕ: необходимо объяснить, почему для представленного метода Гаусса решения различаются.

In [131]:
import numpy as np
import numpy.linalg as la
import copy 

def gauss(A_in, b_in):
    n = b_in.size
    A = copy.deepcopy(A_in)
    b = copy.deepcopy(b_in)

    for k in range(0, n - 1):
        for i in range(k + 1, n):
            if A[i,k] != 0:
                c = A[i,k]/A[k,k]
                A[i, (k + 1):n] = A[i, (k + 1):n] - c * A[k, (k + 1):n]
                b[i] = b[i] - c * b[k]
                
    # Обратный ход
    for k in range(n - 1, -1, -1):
        b[k] = (b[k] - np.dot(A[k, (k + 1):n], b[(k + 1):n]))/A[k,k]
    return b

In [132]:
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]])
b1 = np.array([0., 0., 1.])

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

print('mu1 = ', la.cond(A1))
print('mu2 = ', la.cond(A2))

mu1 =  16.39373162228438
mu2 =  16.393731622284395


In [133]:
print('u1 = ', la.solve(A1, b1))
print('u2 = ',la.solve(A2, b2))

u1 =  [1. 1. 1.]
u2 =  [1. 1. 1.]


In [134]:
print('u1 = ', gauss(A1, b1))
print('u2 = ', gauss(A2, b2))

u1 =  [0.55511151 0.25       0.25      ]
u2 =  [1. 1. 1.]


**Объясним, почему для представленного метода Гаусса значения различаются.**

## Часть 2. LU разложение

Задание: 

Реализовать алгоритм решения предыдущей задачи с матрицей $A_2$ с помощью $LU$-разложение. В решении должна выводиться $L$, $U$ и решение системы. 

ВАЖНО: реализация метода $LU$ должна быть получена путем небольшой модификации метода $\textit{gauss}$!  При этом саму реализацию можно разделить на два метода: один метод собственно находит $LU$ разложение (можно сделать переделкой цикла для матрицы $A$ метода $\textit{gauss}$), второй метод - непосредственное решение системы с помощью прямого и обратного хода. Ни в каком виде нельзя пользоваться пакетными методами (в частности, la.solve)

### LU - разложение с помощью пакета sympy

Чтобы убедиться, что разложение получено верно, можно воспользоваться скриптом ниже

In [135]:
import sympy as sp

A = sp.Matrix([[2., -1., 0.], [-1., 2., -1.], [0, 1., -1.]])
L, U, _ = A.LUdecomposition()
L

Matrix([
[   1,                 0, 0],
[-0.5,                 1, 0],
[   0, 0.666666666666667, 1]])

In [136]:
U

Matrix([
[2.0, -1.0,                0.0],
[  0,  1.5,               -1.0],
[  0,    0, -0.333333333333333]])

In [137]:
def lu_decomposition(A_in, b_in):
    n = b_in.size
    A = copy.deepcopy(A_in)
    
    L = np.eye(n)
    U = np.zeros((n, n))
    
    for k in range(0, n - 1):
        for i in range(k + 1, n):
            if A[i,k] != 0:
                L[i, k] = A[i, k] / A[k, k]  # Заполняем столбцы L
                A[i, k:n] = A[i, k:n] - L[i, k] * A[k, k:n]

    U = np.triu(A)

    return L, U

def lu_solve(L, U, b):
    n = b.size

    # Решаем Lv = b (ПРЯМОЙ ХОД)
    v = np.zeros(n)
    for i in range(n):
        v[i] = b[i] - np.dot(L[i, :i], v[:i])
    
    # Решаем Uu = v (ОБРАТНЫЙ ХОД)
    u = np.zeros(n)
    for k in range(n - 1, -1, -1):
        u[k] = (v[k] - np.dot(U[k, k+1:n], u[k+1:n])) / U[k, k]
    
    return u

L, U = lu_decomposition(A2, b2)

print("L:")
display(sp.Matrix(L))

print("U:")
display(sp.Matrix(U))

print("Решение системы:")
display(sp.Matrix(lu_solve(L, U, b2)))

L:


Matrix([
[    1.0,               0.0, 0.0],
[   -0.5,               1.0, 0.0],
[5.0e-17, 0.666666666666667, 1.0]])

U:


Matrix([
[2.0, -1.0,                0.0],
[0.0,  1.5,               -1.0],
[0.0,  0.0, -0.333333333333333]])

Решение системы:


Matrix([
[1.0],
[1.0],
[1.0]])

## Часть 3. Нахождение обратной матрицы с помощью LU разложения

Задание:

Предложить алгоритм с использованием $LU$-разложения и найти обратную матрицу с точностью $\varepsilon = 10^{-3}$:
$$
A = \begin{pmatrix} 
1 & 1  & 1 \\
0 & 1 & 2 \\
7 & 1 & 4 \\
\end{pmatrix}
$$
Для необходимых оценок использовать первую норму. Сравнить результат со значением, найденным с помощью функции numpy.linalg.inv.

In [138]:
A = sp.Matrix([[1, 1, 1], [0, 1, 2], [7, 1, 4]])

np.linalg.inv(A)

UFuncTypeError: Cannot cast ufunc 'inv' input from dtype('O') to dtype('float64') with casting rule 'same_kind'

## Часть 4. Модифицированный метод Гаусса

Модифицировать метод Гаусса из Части 1 так, чтобы система $A_1x = b_1$ решалась корректно. ВАЖНО: реализация метода должна быть получена путем модификации метода $\textit{gauss}$, а не переписыванием кода с нуля! 